In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
import chromadb
import uuid
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
import re
load_dotenv()

True

In [41]:
# pip install --upgrade --quiet  langchain-google-community

In [2]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
from langgraph.graph import StateGraph
# from langchain_anthropic import ChatAnthropic
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode
from langgraph.graph import MessagesState, StateGraph, START, END

In [3]:
os.environ["GOOGLE_CSE_ID"] =os.getenv("GOOGLE_CSE_ID")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [4]:
llm=ChatGroq(
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="deepseek-r1-distill-llama-70b"
)

In [9]:
response=llm.invoke("who is current president of usa")

In [43]:
from langchain_core.tools import Tool
from langchain_google_community import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

tool = Tool(
    name="google_search",
    description="Search Google for recent results and give consize in 2-3 lines answer.",
    func=search.run,
)

In [51]:
memory = MemorySaver()
model = llm
search = GoogleSearchAPIWrapper()
tools = [search.run]
agent_executor = create_react_agent(model, tools)

In [108]:
# question  ="Hi I am dheeraj"
# ans=tool.run(question)
response=llm.invoke(f"This is recent update I have found through google search Information: {ans}, so just make it to the point and remove unnecessary for this question:{question}")

In [109]:
ans

"Sep 30, 2024 ... I'm Dheeraj from Chennai, India. I've been admitted into L&S this fall. I figured I'll major in economics because I love gambling. Oct 19, 2021 ... 27 votes, 69 comments. Hey Reddit, it's Dheeraj. As developers ourselves, Manoj (DevRev co-founder) and I realized there are a lot of\xa0... Nov 16, 2023 ... I'm a photographer, filmmaker and a digital artist based in India. My work revolves around cityscapes and landscapes. I love architecture, symmetry, color and\xa0... Mar 17, 2016 ... 3.9K votes, 373 comments. Hi Reddit! My research is about what happens to our memories when we cannot remember. When we experience memory\xa0... Aug 27, 2024 ... Hi Dheeraj,. thanks a lot for the additional tests. I am a little lost myself right now to be honest. Have you tried changing the supply\xa0... Jan 31, 2023 ... Hello @Dheeraj,. I am glad this is working now! Thank you @mitchell ... Hi. I'm trying to update multiple columns on the board using. let\xa0... Sep 27, 2017 ... Hi,I am 

In [110]:
response.content

'<think>\nOkay, so I need to figure out how to help this user. They provided a bunch of information from a Google search about someone named Dheeraj. It looks like they\'re trying to get a concise version of this information for a question. The user wrote "Hi I am dheeraj" and wants it to the point without unnecessary details.\n\nFirst, I should go through each of the points they provided. There\'s a lot here, so I need to pick out the key details. Let\'s see:\n\n- Admitted to L&S, majoring in economics because he loves gambling.\n- Co-founder of DevRev with Manoj, mentioned on Reddit.\n- Photographer, filmmaker, digital artist based in India, into cityscapes, architecture, symmetry, color.\n- Research on memory, with a Reddit post about it.\n- Some technical discussions about tests and updating columns.\n- Backend engineer at Marvin, loves backend systems and web-dev.\n- Computer Science grad student, researcher, developer with expertise in Machine Learning.\n- Hacked Facebook and Ins

In [111]:
cresponse=re.sub(r"<think>.*?</think>", "", response.content, flags=re.DOTALL).split("\n")
cresponse

['',
 '',
 "Hi, I'm Dheeraj. I'm a student at L&S, majoring in economics, and I'm passionate about photography, filmmaking, and digital art. I also work as a backend engineer and am interested in Machine Learning."]

In [69]:
# tool.run("who is current president of usa")

In [40]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="who is the president of usa")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

who is the president of usa
================================== Ai Message ==================================
Tool Calls:
  run (call_g4d2)
 Call ID: call_g4d2
  Args:
    query: president of USA
================================= Tool Message =================================
Name: run

America Is Back · Donald J. Trump. President of the United States · JD Vance. VICE PRESIDENT OF THE UNITED STATES · Melania Trump. First Lady OF THE UNITED STATES. The president is one of the world's most powerful political figures and the leader of the world's only remaining superpower. 45th & 47th President of the United States. The Golden Age of America Begins Right Now. Donald J. Trump serves as the 47th President of the United States. He also served as the 45th. Read the President's full biography. 16M Followers, 3 Following, 83 Posts - President Donald J. Trump (@potus) on Instagram: "45th & 47th President of the Unit

In [47]:
tools = []
tool_node = ToolNode(tools)
model = llm
bound_model = model.bind_tools(tools)

In [49]:
@tool
def search(query: str):
    """Call to surf the web."""
    # This is a placeholder for the actual implementation
    # Don't let the LLM know this though 😊
    return "It's sunny in San Francisco, but you better look out if you're a Gemini 😈."
def should_continue(state: MessagesState):
    """Return the next node to execute."""
    last_message = state["messages"][-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return END
    # Otherwise if there is, we continue
    return "action"
# Define the function that calls the model
def call_model(state: MessagesState):
    response = bound_model.invoke(state["messages"])
    # We return a list, because this will get added to the existing list
    return {"messages": response}

In [50]:
# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Next, we pass in the path map - all the possible nodes this edge could go to
    ["action", END],
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile(checkpointer=memory)

In [51]:
config = {"configurable": {"thread_id": "5"}}

In [ ]:
memory = MemorySaver()
query="do you remember my name?"

input_message = HumanMessage(content=f"{query}")
for event in app.stream({"messages": [query]}, config, stream_mode="values"):
    cleaned_response = re.sub(r"<think>.*?</think>", "", event["messages"][-1].content, flags=re.DOTALL)
    print(cleaned_response)
    # event["messages"][-1].pretty_print()

do you remember my name?


Yes, I remember! You mentioned your name is Dheeraj. How can I assist you today? 😊


In [ ]:
print(cleaned_response)

In [18]:
while True:
    query=input("inter your text")
    if query.lower() == "exit":
        break
    input_message = HumanMessage(content=f"{query}")
    for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
        event["messages"][-1].pretty_print()

================================ Human Message =================================

hey ,how are you
================================== Ai Message ==================================

<think>

</think>

Hey! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you with whatever you need. How are *you* doing? 😊
================================ Human Message =================================

I am dheeraj 
================================== Ai Message ==================================

<think>
Alright, the user just said, "I am dheeraj." So, he's introducing himself. I should respond in a friendly and welcoming manner. Maybe start with a greeting and ask how I can assist him today. Keeping it open-ended encourages him to share what he needs help with. I want to make sure he feels comfortable asking for anything. Let's keep the tone warm and approachable.
</think>

Hello, Dheeraj! Welcome. How can I assist you today?
================================ Human M